In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading the csv files
file1 = pd.read_csv('./files_for_lab/csv_files/file1.csv')
file2 = pd.read_csv('./files_for_lab/csv_files/file2.csv')
file3 = pd.read_csv('./files_for_lab/csv_files/file3.csv')

In [3]:
print(file1.shape,file2.shape,file3.shape)

(4008, 11) (996, 11) (7070, 11)


In [4]:
# Converting all strings to lowercase.
file1.columns=list(map(lambda x: x.lower(),file1.columns))
file2.columns=list(map(lambda x: x.lower(),file2.columns))
file3.columns=list(map(lambda x: x.lower(),file3.columns))

In [5]:
file1=file1.rename(columns={'st':'state'})
file2=file2.rename(columns={'st':'state'})
# I think they all have now the same names,
# but in different order, and I'm going to check it.

# I check for every column in file2,file3 if it's in 
# file1.columns. And return true if all are true for each file.
print(all(list(map(lambda x: x in file1.columns,file2.columns))))
print(all(list(map(lambda x: x in file1.columns,file3.columns))))
# It's true for all of them and they all have the same length, so they 
# have the same column names but in a different order.

True
True


In [6]:
# So now I just have to order them the same way as for example file1
cols = list(file1.columns)
file2, file3 = file2[cols], file3[cols]

In [7]:
# I get rid of the %, convert to numeric and divide by 100
file2['customer lifetime value'] = file2['customer lifetime value'].str[:-1]
file2['customer lifetime value'] = pd.to_numeric(file2['customer lifetime value'], errors='coerce')
file2['customer lifetime value'] /= 100
file1['customer lifetime value'] = file1['customer lifetime value'].str[:-1]
file1['customer lifetime value'] = pd.to_numeric(file1['customer lifetime value'], errors='coerce')
file1['customer lifetime value'] /= 100

In [8]:
# Now I concatenate the data
data = pd.concat([file1,file2,file3],axis=0)

In [9]:
# checking the numerical data
data._get_numeric_data().head(5)

,customer lifetime value,income,monthly premium auto,total claim amount
0,NaN,0.0,1000.0,2.704934
1,6979.5359,0.0,94.0,1131.464935
2,12887.4317,48767.0,108.0,566.472247
3,7645.8618,0.0,106.0,529.881344
4,5363.0765,36357.0,68.0,17.269323


In [10]:
# checking the categorical data
data.select_dtypes(['object']).head(5)

,customer,state,gender,education,number of open complaints,policy type,vehicle class
0,RB50392,Washington,NaN,Master,1/0/00,Personal Auto,Four-Door Car
1,QZ44356,Arizona,F,Bachelor,1/0/00,Personal Auto,Four-Door Car
2,AI49188,Nevada,F,Bachelor,1/0/00,Personal Auto,Two-Door Car
3,WW63253,California,M,Bachelor,1/0/00,Corporate Auto,SUV
4,GA49547,Washington,M,High School or Below,1/0/00,Personal Auto,Four-Door Car


In [11]:
# We see that the attribute "number of open complaints" it's messed up
# so I guess that's why we drop it
data=data.drop(['education','number of open complaints'],axis=1)

In [12]:
#showing duplicated rows
data[data.duplicated()]

,customer,state,gender,customer lifetime value,income,monthly premium auto,policy type,vehicle class,total claim amount
1072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,GS98873,Arizona,F,3239.1247,16061.0,88.0,Personal Auto,Four-Door Car,633.6
1,CW49887,California,F,4626.8011,79487.0,114.0,Special Auto,SUV,547.2


In [13]:
# Out of the 2939 duplicated rows, 2937 are blank rows.
print(len(data[data.isna().all(axis=1)]))
# I remove then 
data = data.drop_duplicates()

2937


In [14]:
# We have at most 123 nulls in gender, but it's a small percent ~ 1%
# since there's a total of 9135 rows. 
data.isna().sum()
# I will fill those values with the mode just as we did in class
# to practice it.

customer                     1
state                        1
gender                     123
customer lifetime value      8
income                       1
monthly premium auto         1
policy type                  1
vehicle class                1
total claim amount           1
dtype: int64

In [15]:
data['gender'] = data['gender'].fillna(data['gender'].mode()[0])

In [16]:
# we now can drop all the NaN values since they're just ~0,01%.
data = data.dropna()

In [17]:
# And now I filter out the data whose income is less or equal to 0
data = data[data['income'] >0 ]
data = data.reset_index(drop=True)
data

,customer,state,gender,customer lifetime value,income,monthly premium auto,policy type,vehicle class,total claim amount
0,AI49188,Nevada,F,12887.431700,48767.0,108.0,Personal Auto,Two-Door Car,566.472247
1,GA49547,Washington,M,5363.076500,36357.0,68.0,Personal Auto,Four-Door Car,17.269323
2,OC83172,Oregon,F,8256.297800,62902.0,69.0,Personal Auto,Two-Door Car,159.383042
3,XZ87318,Oregon,F,5380.898600,55350.0,67.0,Corporate Auto,Four-Door Car,321.600000
4,DY87989,Oregon,M,24127.504000,14072.0,71.0,Corporate Auto,Four-Door Car,511.200000
...,...,...,...,...,...,...,...,...,...
6830,TF56202,California,M,5032.165498,66367.0,64.0,Personal Auto,Two-Door Car,307.200000
6831,YM19146,California,F,4100.398533,47761.0,104.0,Personal Auto,Four-Door Car,541.282007
6832,LA72316,California,M,23405.987980,71941.0,73.0,Personal Auto,Four-Door Car,198.234764
6833,PK87824,California,F,3096.511217,21604.0,79.0,Corporate Auto,Four-Door Car,379.200000


In [31]:
# I'm not sure if I'm supposed to do this but I'll also clean
# so that every state or gender has only one string associated.
print(data['state'].value_counts())
print(data['gender'].value_counts())

California    2250
Oregon        1937
Arizona       1217
Nevada         665
Washington     593
Cali            96
AZ              56
WA              21
Name: state, dtype: int64
F         3555
M         3221
Male        27
female      20
Femal       12
Name: gender, dtype: int64


In [38]:
#defining functions to use with map
def clean_state(st):
    if st.startswith('Cali'):
        return 'California'
    elif st.startswith('W'):
        return 'Washington'
    elif st.startswith('AZ'):
        return 'Arizona'
    else:
        return st

def clean_gender(gnd):
    if gnd.startswith('f') or gnd.startswith('F'):
        return 'Female'
    else:
        return 'Male'

In [39]:
data['state'] = list(map(clean_state,data['state']))
data['gender'] = list(map(clean_gender,data['gender']))

In [40]:
print(data['state'].value_counts())
print(data['gender'].value_counts())

California    2346
Oregon        1937
Arizona       1273
Nevada         665
Washington     614
Name: state, dtype: int64
Female    3587
Male      3248
Name: gender, dtype: int64
